In [30]:
from events_data import CompetitionData, events, maximize_events, events_in_groups
from marks_reader import read_marks
from ponderator import ponderate_marks, ponderate_events_marks
from simulator import simulate_all
from prediction_saver import save_prediction

In [27]:
event_folder = 'classified'

only = ["atl_jab"]

olimpicsData = CompetitionData(events, maximize_events, events_in_groups)

In [28]:
data = read_marks(event_folder, only)

In [29]:
pond_times = {
    2022: 4,
    2021: 2,
    2020: 1,
}

ponderated_data = ponderate_events_marks(data, olimpicsData, pond_times)

In [5]:
pred = simulate_all(ponderated_data, times=1000, top=100, warnings=True, only=only)

Simulating event: Lanzamiento de la jabalina. Sex: male
Calculating models
(1.32 %) Creating kde model of USA_marc_anthony_minichello...
(2.63 %) Creating kde model of BEL_timothy_herman...
(3.95 %) Creating kde model of GER_johannes_vetter...
(5.26 %) Creating kde model of CZE_jakub_vadlejch...
(6.58 %) Creating kde model of GER_julian_weber...
(7.89 %) Creating kde model of USA_tzuriel_pedigo...
(9.21 %) Creating kde model of GRE_dimitris_tsitsos...
(10.53 %) Creating kde model of TTO_keshorn_walcott...
(11.84 %) Creating kde model of BAH_keyshawn_strachan...
(13.16 %) Creating kde model of FIN_topias_laine...
(14.47 %) Creating kde model of GER_maurice_voigt...
(15.79 %) Creating kde model of MDA_andrian_mardare...
(17.11 %) Creating kde model of BIH_dejan_mileusnić...
(18.42 %) Creating kde model of COL_arley_ibarguen...
(19.74 %) Creating kde model of USA_donavon_banks...
(21.05 %) Creating kde model of BRA_pedro_henrique_rodrigues...
(22.37 %) Creating kde model of SVK_maximilian

In [8]:
save_prediction(pred, top=15)

### OPTIMIZATION SECTION

In [21]:
from optimizer import create_dataset

x, y = create_dataset(ponderated_data)

In [78]:
from optimizer import SMAC4BBOptimizer

smac = SMAC4BBOptimizer(50)
print(smac.optimize(x['atl_jab']['male'], y['atl_jab']['male'], olimpicsData.is_maximize_event('atl_jab')))

Configuration(values={
  'bandwidth': 4.655816644000222,
  'n_sim': '1500',
  'n_years': 2,
})



In [53]:
from autogoal.ml import AutoML
from autogoal.grammar import ContinuousValue
from autogoal.kb import *   
from autogoal.ml import AutoML

import numpy as np


class NewAlgorithm(AlgorithmBase):
    def __init__(
        self,
        bandwith: ContinuousValue(min=0.1, max=5),
        ):
        self.bandwith = bandwith

    def run(self, input: MatrixContinuousDense) -> float:
        pond_times = {
            2022: 4,
            2021: 2,
            2020: 1,
        }
        # TODO: fix maximize
        #pond_results = ponderate_marks(input, True, pond_times)
        #final_results = pond_results.Result.to_numpy()
        

        model = KernelDensity(bandwidth=self.bandwith, kernel='gaussian')
        model.fit(X = input)

        return model.sample()[0][0]


automl = automl = AutoML(
    input=(MatrixContinuousDense, Supervised[float]), 
    output=float,
    registry=[NewAlgorithm],
)

nx = [a.Result.to_numpy().reshape((-1,1)) for a in x['atl_jab']['female']]

automl.fit(nx, y['atl_jab']['female'])

/home/karla/.pyenv/versions/3.7.2/lib/python3.7/site-packages/autogoal/kb/_algorithm.py:272: UserWarning: No step answered message train.
  warnings.warn(f"No step answered message {msg}.")
/home/karla/.pyenv/versions/3.7.2/lib/python3.7/site-packages/autogoal/kb/_algorithm.py:272: UserWarning: No step answered message train.
  warnings.warn(f"No step answered message {msg}.")
/home/karla/.pyenv/versions/3.7.2/lib/python3.7/site-packages/autogoal/kb/_algorithm.py:272: UserWarning: No step answered message train.
  warnings.warn(f"No step answered message {msg}.")
/home/karla/.pyenv/versions/3.7.2/lib/python3.7/site-packages/autogoal/kb/_algorithm.py:272: UserWarning: No step answered message train.
  warnings.warn(f"No step answered message {msg}.")
/home/karla/.pyenv/versions/3.7.2/lib/python3.7/site-packages/autogoal/kb/_algorithm.py:272: UserWarning: No step answered message train.
  warnings.warn(f"No step answered message {msg}.")
/home/karla/.pyenv/versions/3.7.2/lib/python3.7/si

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (73,) + inhomogeneous part.

In [ ]:
# # Tratamiento de datos
# # ==============================================================================
# import numpy as np
# import joblib

# # Gráficos
# # ==============================================================================
# import matplotlib.pyplot as plt
# from matplotlib import style

# # Ajuste de distribuciones
# # ==============================================================================
# from sklearn.neighbors import KernelDensity

# # Configuración matplotlib
# # ==============================================================================
# plt.rcParams['figure.figsize'] = (12,8)
# style.use('ggplot') or plt.style.use('ggplot')

# # Configuración warnings
# # ==============================================================================
# import warnings
# warnings.filterwarnings('ignore')

# from concurrent.futures import *

In [ ]:
# def single_model(event_name, sex, name, save_folder, models):
#     model = None
#     if save_folder is not None:
#         folder_path = Path(save_folder)
#         folder_path.mkdir(parents=True, exist_ok=True)
#         full_path = folder_path / f'{name}.pkl'
#         if full_path.exists():
#             model = joblib.load(full_path)
#         else:
#             model = athlete_kde_model(event_name, sex, name)
#             joblib.dump(model, full_path)
#     else:
#         model = athlete_kde_model(event_name, sex, name)
#     models.append(model)

In [ ]:
# def density_pred(kde_model: KernelDensity, samples):    
#     log_density_pred = kde_model.score_samples(X=samples.reshape(-1, 1))
#     return np.exp(log_density_pred)

    
# def plot_model(event, sex, model_name, bounds, Show=True):
#     model: KernelDensity = joblib.load(f'models/{event}/{sex}/{model_name}.pkl')
#     # X = np.linspace(bounds[0], bounds[1], 200)
#     # Y = density_pred(model, X)
#     X, Y = model.evaluate()
#     plt.plot(X, Y, label=model_name)
#     if Show:
#         plt.show()

# bounds = (1700, 1900)
# event, sex = 'atl_jab', 'female'
# plot_model(event, sex, 'AUS_alexandra_roberts', bounds, False)
# # plot_model(event, sex, 'BRN_kalkidan_gezahegne', bounds, False)
# # plot_model(event, sex, 'NED_sifan_hassan', bounds, False)
# plt.legend()

# plt.show()

# model: KernelDensity = joblib.load(f'models/{event}/{sex}/NED_sifan_hassan.pkl')
# print(model.bandwidth)

In [ ]:
# entry = load_entry_json()

# new_entry = {}

# for event in entry:
#     new_entry[event] = {}
#     for sex in entry[event]:
#         new_entry[event][sex] = []
        
#         for name in entry[event][sex]:
#             new_entry[event][sex].append(name.casefold())

# save_prediction(new_entry, "entry_list.json")